<a href="https://colab.research.google.com/github/ganjire/ML2_Project/blob/main/ML2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pickle
import cv2
!pip install keras
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential